In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader, TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [2]:
from langchain_community.embeddings import GPT4AllEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

### `DocumnetManager` & `EmbeddingManager`

In [3]:
class DocumentManager:
    def __init__(self, file_path):
        self.file_path = file_path
        self.document = None
        self.all_sections = []
    def load_documents(self):
        with open(self.file_path, "r", encoding='utf-8') as file:
            self.document=file.read()

    def split_documents(self):
        headers_to_split_on = [("#", "Header 1"), ("##", "Header 2"), ("###", "Header 3"), ("####", "Header 4")]
        header_dict = {"Header 1": "#", "Header 2": "##", "Header 3": "###", "Header 4": "####"}
        text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
        sections = text_splitter.split_text(self.document)
        for section in sections:
            add_header = ""
            for k, v in section.metadata.items():
                add_header += (header_dict[k] + v + '\n')
            section.page_content = add_header + section.page_content
                
        self.all_sections.extend(sections)

        

In [4]:
class EmbeddingManager:
    def __init__(self, all_sections, persist_directory='./vectordb'):
        self.all_sections = all_sections
        self.persist_directory = persist_directory
        self.vectordb = None
    
    def create_and_persist_embeddings(self):
        embedding = HuggingFaceEmbeddings(model_name='C://Users//AFOC//Desktop//llm_mark2//models//multilingual-e5-large-instruct')
        self.vectordb = Chroma.from_documents(documents=self.all_sections, embedding=embedding, persist_directory=self.persist_directory)

### `.md` -> `vector db`(변환)

In [5]:
doc_manager = DocumentManager('C://Users//AFOC//Desktop//llm_mark2//langchain-streamlit//md_files//document1.md')
doc_manager.load_documents()
doc_manager.split_documents()

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//AFOC//Desktop//llm_mark2//langchain-streamlit//md_files//document1.md'

In [ ]:
embed_manager = EmbeddingManager(doc_manager.all_sections)
embed_manager.create_and_persist_embeddings()

### `vectordb` 불러오기 & `retriever` 테스트

In [11]:
persist_directory = 'C://Users//AFOC//Desktop//llm_mark2//langchain-streamlit//vectordb'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=HuggingFaceEmbeddings(model_name='C://Users//AFOC//Desktop//llm_mark2//models//multilingual-e5-large-instruct'))

In [12]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [15]:
docs = retriever.invoke("대학 졸업예정으로 군대연기하려면?")

In [16]:
docs

[Document(metadata={'Header 1': '현역병 입영일자 연기', 'Header 2': '입영일자 연기사유, 연기기간 및 구비서류', 'Header 3': '현역병 입영일자 연기사유별 연기기간 및 첨부서류', 'Header 4': '연기사유 7. **각급학교 입학시험 응시-3**'}, page_content='#현역병 입영일자 연기\n##입영일자 연기사유, 연기기간 및 구비서류\n###현역병 입영일자 연기사유별 연기기간 및 첨부서류\n####연기사유 7. **각급학교 입학시험 응시-3**\n- 연기기간:\n- 대학 편입학 예정자는 대학 수료 또는 졸업 다음 해 5월 말일까지의 범위에서 기간을 정하여 연기.\n- 구비서류\n- 신청인 제출서류: 졸업·수료(예정) 증명서, 수학능력시험 접수증 등.\n- 담당자 확인사항: 없음.'),
 Document(metadata={'Header 1': '현역병 입영일자 연기', 'Header 2': '입영일자 연기사유, 연기기간 및 구비서류', 'Header 3': '현역병 입영일자 연기사유별 연기기간 및 첨부서류', 'Header 4': '연기사유 18. **졸업예정자**'}, page_content='#현역병 입영일자 연기\n##입영일자 연기사유, 연기기간 및 구비서류\n###현역병 입영일자 연기사유별 연기기간 및 첨부서류\n####연기사유 18. **졸업예정자**\n- 연기기간:\n- 각급학교(2년제 대학 이상) 졸업 예정자로서 학업을 계속 희망하는 사람은 각급 학교별 제한 연령 초과 1년의 범위에서 졸업 시까지 연기(휴학자 제외).\n- 제한 연령 기준:\n- 2년제 대학: 23세.\n- 3년제 대학: 24세.\n- 4년제 대학: 25세.\n- 5년제 대학: 26세.\n- 6년제 대학: 27세.\n- 의과·약학 대학: 28세.\n- 석사학위 2년제 과정: 27세.\n- 석사학위 2년 초과 과정: 28세.\n- 의과·약학 대학원: 29세.\n- 박사학위 과정: 29세.\n- 연수기관: 27세.\n- 구비